**Colour Histogram**

In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt
import glob
import numpy as np
from numpy.linalg import norm
from tqdm.notebook import tqdm
%matplotlib inline
path_query = 'datasets/query/'
path_query_txt = 'datasets/query_txt/'
path_gallery = 'dataset/gallery/'

name_query = glob.glob(path_query+'*.jpg')
num_query = len(name_query)
name_gallery = glob.glob(path_gallery+'*.jpg')
num_gallery = len(name_gallery)

def crop(path):
    img = cv.imread(path)
    txtIndex = path.split('.')[0].split('/')
    content = []

    with open('datasets/query_txt/' + txtIndex[-1] + ".txt", 'r') as f:
        content = f.read().split()

    x = int(content[0])
    y = int(content[1])
    w = int(content[2])
    h = int(content[3])
    img = img[y : y + h, x : x + w]
    return img

def widthAndHeight(path):
    txtIndex = path.split('.')[0].split('/')
    content = []

    with open('datasets/query_txt/' + txtIndex[-1] + ".txt", 'r') as f:
        content = f.read().split()

    w = int(content[2])
    h = int(content[3])
    return (w, h)

def HSVHistogram(img):
    Hist = []
    hue = np.zeros((180), dtype=int)
    saturation = np.zeros((256), dtype=int)
    value = np.zeros((256), dtype=int)

    height = img.shape[0]
    width = img.shape[1]

    for j in range(height):
        for k in range(width):
            h = img[j][k][0]
            s = img[j][k][1]
            v = img[j][k][2]
            hue[h] += 1
            saturation[s] += 1
            value[v] += 1

    cv.normalize(hue, hue, alpha=2500, beta=0, norm_type=cv.NORM_L2)
    cv.normalize(saturation, saturation, alpha=2500, beta=0, norm_type=cv.NORM_L2)
    cv.normalize(value, value, alpha=2500, beta=0, norm_type=cv.NORM_L2)

    Hist.extend(hue)
    Hist.extend(saturation)
    Hist.extend(value)
    return Hist

def sliding_window(image, stepSize, windowSize):
	for y in range(0, image.shape[0] - windowSize[1], stepSize):
		for x in range(0, image.shape[1] - windowSize[0], stepSize):
			yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])
    

In [ ]:
rank_allQuery = np.zeros((num_query, 10))

for n in range(num_query):
    similarity = []
    bgr_qry = crop(name_query[n])
    (winW, winH) = widthAndHeight(name_query[n])
    img_qry = cv.cvtColor(bgr_qry, cv.COLOR_BGR2HSV)
    Hist_qry = HSVHistogram(img_qry)

    for i in tqdm(range(num_gallery)):
        bgr = cv.imread(name_gallery[i], cv.IMREAD_COLOR)
        img = cv.cvtColor(bgr, cv.COLOR_BGR2HSV)
        sim_single = []

        for (x, y, window) in sliding_window(img, stepSize=64, windowSize=(winW, winH)):
            if window.shape[0] != winH or window.shape[1] != winW:
                continue

            Hist_gal = HSVHistogram(window)
            cos_sim = np.inner(Hist_qry, Hist_gal)/(norm(Hist_qry)*norm(Hist_gal))
            sim_single.append(cos_sim)

        max_sim = max(sim_single, default = 0)
        similarity.append(max_sim)
    descend_index = sorted(range(len(similarity)), key = lambda k: similarity[k], reverse=True)
    descend_name_gallery = [name_gallery[index].split('.')[0].split('/')[-1] for index in descend_index[0:10]]
    rank_allQuery[n, :] = descend_name_gallery

f = open('rankList_hsv.txt', 'w')
for i in range(num_query):
    f.write('Query '+str(i + 1)+': ')
    for j in range(10):
        f.write(str(np.int32(rank_allQuery[i, j])) + ' ')
    f.write('\n')
f.close()


**Convolutional Neural Networks (VGG-16)**

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
from pathlib import Path
from PIL import Image
import numpy as np
import glob
import cv2 as cv
from tqdm.notebook import tqdm

class FeatureExtractor:
    def __init__(self):
        base_model = VGG16(weights='imagenet')
        self.model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)
    def extract(self, img):
        img = img.resize((224, 224))
        img = img.convert('RGB')
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        feature = self.model.predict(x)[0]
        return (feature / np.linalg.norm(feature))

In [ ]:
path_query = 'drive/MyDrive/datasets/query/'
path_query_txt = 'drive/MyDrive/datasets/query_txt/'
path_gallery = 'drive/MyDrive/datasets/gallery/'

name_query = glob.glob(path_query+'*.jpg')
num_query = len(name_query)
name_gallery = glob.glob(path_gallery+'*.jpg')
num_gallery = len(name_gallery)

fe = FeatureExtractor()
for i in tqdm(range(num_gallery)):
    img_gal = Image.open(name_gallery[i])
    feature = fe.extract(img=img_gal)
    feature_path = 'drive/MyDrive/datasets/gallery_feature/' + name_gallery[i].split('.')[0].split('/')[-1] + '.npy'
    np.save(feature_path, feature)

In [ ]:
rank_allQuery = np.zeros((num_query, 10))

for n in range(num_query):
    dist = []
    img_qry = Image.open(name_query[n])
    feat_qry = fe.extract(img=img_qry)

    for i in tqdm(range(num_gallery)):
        feat_gal = np.load('drive/MyDrive/datasets/gallery_feature/'+name_gallery[i].split('.')[0].split('/')[-1]+'.npy')
        distance = np.linalg.norm(feat_gal - feat_qry, axis=0)
        dist.append(distance)
    ascend_index = sorted(range(len(dist)), key = lambda k: dist[k])
    ascend_name_gallery = [name_gallery[index].split('.')[0].split('/')[-1] for index in ascend_index[0:10]]
    rank_allQuery[n, :] = ascend_name_gallery

f = open('rankList_CNN.txt', 'w')
for i in range(num_query):
    f.write('Query '+str(i + 1)+': ')
    for j in range(10):
        f.write(str(np.int32(rank_allQuery[i, j])) + ' ')
    f.write('\n')
f.close()

**VGG-16 and Scale-Invariant Feature Transform**

In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt
import glob
import numpy as np
%matplotlib inline
path_query = 'drive/MyDrive/datasets/query/'
path_query_txt = 'drive/MyDrive/datasets/query_txt/'
path_gallery = 'drive/MyDrive/datasets/gallery/'

name_query = glob.glob(path_query+'*.jpg')
num_query = len(name_query)
name_gallery = glob.glob(path_gallery+'*.jpg')
num_gallery = len(name_gallery)

def crop(path):
    img = cv.imread(path)
    txtIndex = path.split('.')[0].split('/')
    content = []

    with open(path_query_txt + txtIndex[-1] + ".txt", 'r') as f:
        content = f.read().split()

    x = int(content[0])
    y = int(content[1])
    w = int(content[2])
    h = int(content[3])
    img = img[y : y + h, x : x + w]
    return img


In [ ]:
sift = cv.SIFT_create()
for i in tqdm(range(num_gallery)):
    img_gal = cv.imread(name_gallery[i])
    gray_gal = cv.cvtColor(img_gal, cv.COLOR_BGR2GRAY)
    kp_gal, des_gal = sift.detectAndCompute(gray_gal, None)
    feature_path = 'drive/MyDrive/datasets/gallery_feature_SIFT/' + name_gallery[i].split('.')[0].split('/')[-1] + '.npy'
    np.save(feature_path, des_gal)

In [ ]:
rank_allQuery = np.zeros((num_query, 10))
sift = cv.SIFT_create()
fe = FeatureExtractor()
for n in range(num_query):
    dist = []
    img_qry = Image.open(name_query[n])
    feat_qry = fe.extract(img=img_qry)

    img_qry = crop(name_query[n])
    gray_qry = cv.cvtColor(img_qry, cv.COLOR_BGR2GRAY)
    kp_qry, des_qry = sift.detectAndCompute(gray_qry, None)

    for i in tqdm(range(num_gallery)):
        feat_gal = np.load('drive/MyDrive/datasets/gallery_feature/'+name_gallery[i].split('.')[0].split('/')[-1]+'.npy')
        Distance = np.linalg.norm(feat_gal - feat_qry, axis=0)

        des_gal = np.load('drive/MyDrive/datasets/gallery_feature_SIFT/'+name_gallery[i].split('.')[0].split('/')[-1]+'.npy')
        bf = cv.BFMatcher(cv.NORM_L2, crossCheck=True)
        matches = bf.match(des_gal, des_qry)
        num_matches = len(matches)
        dist_list = [DMatch.distance for DMatch in matches]
        dist_l2 = sum(dist_list) / num_matches
        dist.append(dist_l2 + Distance)

    ascend_index = sorted(range(len(dist)), key = lambda k: dist[k])
    ascend_name_gallery = [name_gallery[index].split('.')[0].split('/')[-1] for index in ascend_index[0:10]]
    rank_allQuery[n, :] = ascend_name_gallery

f = open('drive/MyDrive/rankList_CNN+SIFT.txt', 'w')
for i in range(num_query):
    f.write('Query '+str(i + 1)+': ')
    for j in range(10):
        f.write(str(np.int32(rank_allQuery[i, j])) + ' ')
    f.write('\n')
f.close()